## Queries - ComCam

In this notebook, we show how to query the ComCam repository\
and view the resulting images.\
This notebook uses the most recent calibrations
Craig Lage - 30-Aug-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

In [ ]:
butler = Butler('/repo/embargo', collections=["LSSTComCam/raw/all", "LSSTComCam/calib",
                                              "u/abrought/LSSTComCam/calibs/w_2024_28/bias.08132024a",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/defects.08132024a",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/dark.08132024c",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/flat_r03.08132024a",
                                             "u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a",
                                                ])

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20240808
instrument = "LSSTComCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


# Get the data from the headers

In [ ]:
expId = 2024080800029
mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
for key in mData.keys():
    print(key, mData[key])

# Now get the raw data

## Next, look at the raw data from one of the exposures.
### Because of the large pedestal, we don't see much.  We need to do ISR

In [ ]:
expId = 2024080800029
raw = butler.get('raw', detector=4, exposure=expId, instrument=instrument)


## Some people have expressed the desire to download the FITS files.  This cell will do that.  

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/2024080800029_4_raw.fits"
raw.image.writeFits(filename)

In [ ]:
%matplotlib inline        
x = plot(raw, stretch='ccs')

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2024080800029
exp = butler.get('raw', detector=2, exposure=expId, instrument=instrument)
biasExp = butler.get('bias', detector=2, exposure=expId, instrument=instrument) # This is a bias image associated with the data
ptc = butler.get('ptc', detector=2, instrument=instrument,
                 collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='linear')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_{expId}.png")

# Plot a small region

In [ ]:
x = plot(isrResult.exposure.image.array[2200:2300, 2600:2700])

# Get the value of a particular pixel

In [ ]:
isrResult.exposure.image.array[2100, 2100]

# Plot the cross-chip uniformity

In [ ]:
plt.title(f"ComCam {expId}, Bias")
plt.plot(isrResult.exposure.image.array[2100, :], label='X cut')
plt.plot(isrResult.exposure.image.array[:, 2100], label='Y cut')
plt.ylim(0,50000)
#plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slices_{expId}.png")

## Some people have expressed the desire to download the FITS files.  This cell will do that.  

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/2024080800029_4_isr.fits"
isrResult.exposure.image.writeFits(filename)

# Now assemble all 9 CCDs and plot the result

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    biasExp = imageSource.butler.get('bias', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    isrResult = isrTask.run(exp, bias=biasExp)
    oim = isrResult.exposure.image
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTComCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
import lsst.afw.display as afwDisplay
disp = afwDisplay.Display(1, "matplotlib")
#disp.scale('linear', min = 45000, max=65000)
disp.scale('linear', min='zscale')
dayObs = 20240808
seqNum = 29
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft="R22",
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=myCallback,
                                                       background=np.nan),
                              binSize=1, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Stuttered_{dayObs}_{seqNum}.png")

# Plot the cross-detector uniformity

In [ ]:
%matplotlib inline
plt.title(f"ComCam {dayObs}_{seqNum}")
plt.plot(mos.array[6000, :], label="X slice")
plt.plot(mos.array[:, 6000], label="Y slice")
plt.ylim(0, 60000)
#plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slices_{dayObs}_{seqNum}.png")

# Plot a small region

In [ ]:
x = plot(mos.array[6000:6100, 6000:6100])

# The cell below will save the combined image as a FITS file

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/FITS_2024080800029.fits"
mos.writeFits(filename)

In [ ]:
type(raw.image)

# Variability per amp

In [ ]:
%matplotlib inline
clip = 10.0
expId = 2024071200007

for det in range(9):
    exp = butler.get('raw', detector=det, exposure=expId)
    biasExp = butler.get('bias', detector=det, exposure=expId) # This is a bias image associated with the data
    isrResult = isrTask.run(exp, bias=biasExp) # This runs the ISR
    for amp in exp.getDetector().getAmplifiers():
        data = isrResult.exposure[amp.getBBox()].image.array.flatten()
        data = (data - np.median(data)) / np.median(data) * 100.0
        sortedData = np.sort(data)
        sortedTrimmedData = np.clip(sortedData, np.percentile(sortedData, clip), np.percentile(sortedData, 100 - clip))
        plt.plot(sortedTrimmedData)
plt.ylim(-5.0, 5.0)
plt.title(f"ComCam {expId}, per amp variability - 10->90 percentiles")
plt.xlabel("Pixel count")
plt.ylabel("Percent variation from Median")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Amp_Nonuniformity_{expId}.png")